In [132]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import re 

# Keras 2.2.4/tensorflow 1.14.0 -> Version matching for local machine

from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from keras import initializers

myname = 'Gagandeep'

files = []
import os
for dirname, _, filenames in os.walk('gagan_Oct_24_2019_data/'):
    for filename in filenames:
#         print(os.path.join(dirname, filename))
        files.append(os.path.join(dirname, filename))

file_list = []
unique_file_list = []    


for value in files:
    x = re.search(".*28_poly_perf.csv$",value)
#     x = re.search(a,value)
    if(x):
#         print(x.group(0))
        file_list.append(x.group(0))
    y = re.search("\w+/\w+/",value)
    if(y):
        if y.group(0) not in unique_file_list:
            unique_file_list.append(y.group(0))

file_list.sort()        
# print(file_list)

In [133]:
# df = pd.DataFrame()
# # print(df)
# for diff_file in file_list:
# #     print(diff_file)
#     train_df = pd.read_csv(diff_file,header=None)
#     df = pd.concat([df, train_df], ignore_index=True)
# #     print(df)
# #     break
# print(df.shape)

In [134]:
# Preprocessing the data
# lower_bound = 0.99
# upper_bound = 1.01
seqorpar =1
arr = []
for diff_file in file_list:
# for layer in range(1,20):
    train_df = pd.read_csv(diff_file,header=None)
#     train_df = pd.read_csv('gagan_Oct_24_2019_data/resnet/resnet_'+str(layer)+'_'+str(seqorpar)+'_poly_perf.csv',header=None)
#     myname = str(layer)+'_'+str(seqorpar)+'_poly_perf.csv'
    train_df = train_df.drop([0,2,3,4,5,6,7,8,9], axis=1)
#     print(train_df)
    train_df = train_df.sort_values(by=[1],ascending=False)
#     print(train_df)
    train_df = train_df.dropna(axis=1)
    npdata = np.array(train_df)
    for i in range(len(npdata)):
        for j in range(len(npdata)):
            arr.append(np.concatenate((npdata[i],npdata[j]), axis=0))
arr = np.asarray(arr)

ver_2_GF = npdata.shape[1]


# 1 x>y
# 2 x<y
# 3 in range


out1 = []
out2 = []
out3 = []
out4 = []
for i in range(len(arr)):
    if arr[i][0] >= arr[i][ver_2_GF]:
        out1.append(1)
        out2.append(0)

    else:
        out1.append(0)
        out2.append(1)

# print(arr[55])

arr = pd.DataFrame(arr)
arr = arr.drop([0,ver_2_GF],axis=1)
arr["sum"] = arr.sum(axis=1)
arr = arr.loc[:,[1,2,3,4,6,7,8,9]].div(arr["sum"], axis=0)
arr.head()

out1 = pd.DataFrame(out1)
out2 = pd.DataFrame(out2)
out3 = pd.DataFrame(out3)

arr = pd.concat([arr, out1, out2,], axis=1, sort=False)
arr = np.array(arr)
print(arr.shape)

(45052, 10)


In [135]:
# Till Here I have preprocessed the data properly.
np.random.shuffle(arr)
length = int(0.70*(len(arr)))
training_set = arr[:length]
test_set = arr[length:]

In [136]:
#tried but deproving# you can add initialization parameter in each layer , it increases performance drastically. ****
#DONE# batch size in powers of 2 like 8,16 or 32. **
#DONE# Can add one more layer with activation function maybe tanh. ***
#DONE# Last layer should be softmax thats good.
#DONE# Optimizer can be adam or sgd.
# can also add a learning rate with optimizer & try to tune learning rate,increases performance drastically. ****
# model = Sequential()
# model.add(Dense(16, input_dim=8, activation='relu'))
# # model.add(Dense(32, activation='relu'))
# # model.add(Dense(16, activation='softsign'))
# model.add(Dense(8, activation='relu'))
# model.add(Dense(2, activation='softmax'))

# # compile the keras model
# # adam = optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# # fit the keras model on the dataset
# model.fit(training_set[:,:-2],training_set[:,8:], epochs=10, batch_size=8)
# # evaluate the keras model
# _, accuracy = model.evaluate(test_set[:,:-2],test_set[:,8:])
# print('Accuracy: %.2f' % (accuracy*100))

from numpy.random import seed
seed(1)
import tensorflow
tensorflow.random.set_seed(2)

# from tensorflow import set_random_seed
# set_random_seed(2)

# model = Sequential()
# initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=2)
# model.add(Dense(32, input_dim=8, activation='relu',kernel_initializer='random_uniform'))
# model.add(Dense(20, activation='relu',kernel_initializer='random_uniform'))
# model.add(Dense(16, activation='relu',kernel_initializer='random_uniform'))
# model.add(Dense(12, activation='softsign',kernel_initializer='random_uniform'))
# model.add(Dense(8, activation='relu',kernel_initializer='random_uniform'))
# model.add(Dense(2, activation='softmax',kernel_initializer='random_uniform'))


model = Sequential()
model.add(Dense(64, input_dim=8, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(16, activation='softsign'))
model.add(Dense(8, activation='relu'))
model.add(Dense(2, activation='softmax'))


# compile the keras model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(training_set[:,:-2],training_set[:,8:], epochs=40, batch_size=18)
# evaluate the keras model
_, accuracy = model.evaluate(test_set[:,:-2],test_set[:,8:])
print('Accuracy: %.2f' % (accuracy*100))


Epoch 1/40
1752/1752 [==============================] - 2s 1ms/step - loss: 0.5806 - accuracy: 0.6817
Epoch 2/40
1752/1752 [==============================] - 2s 1ms/step - loss: 0.5614 - accuracy: 0.6956
Epoch 3/40
1752/1752 [==============================] - 2s 1ms/step - loss: 0.5558 - accuracy: 0.7006
Epoch 4/40
1752/1752 [==============================] - 2s 1ms/step - loss: 0.5510 - accuracy: 0.7029
Epoch 5/40
1752/1752 [==============================] - 2s 1ms/step - loss: 0.5459 - accuracy: 0.7049
Epoch 6/40
1752/1752 [==============================] - 2s 1ms/step - loss: 0.5416 - accuracy: 0.7098
Epoch 7/40
1752/1752 [==============================] - 2s 1ms/step - loss: 0.5365 - accuracy: 0.7111
Epoch 8/40
1752/1752 [==============================] - 2s 1ms/step - loss: 0.5319 - accuracy: 0.7157
Epoch 9/40
1752/1752 [==============================] - 2s 1ms/step - loss: 0.5280 - accuracy: 0.7162
Epoch 10/40
1752/1752 [==============================] - 2s 1ms/step - loss: 0.525

In [137]:
weights = model.get_weights()
print(weights)

[array([[ 5.02305627e-01, -3.69891691e+00,  2.31153443e-01,
        -3.92944407e+00, -1.20478320e+00,  5.44070542e-01,
        -4.56197834e+00, -1.94133580e-01,  6.58595681e-01,
        -2.17431593e+00,  1.55126584e+00,  1.50313675e-01,
         7.20752239e-01, -3.30570555e+00, -2.54958439e+00,
        -2.40634155e+00,  5.92288338e-02, -1.11215830e+00,
        -2.01176596e+00, -5.28685987e-01,  2.62549138e+00,
        -1.64995813e+00, -2.30069876e+00, -2.01566124e+00,
        -2.02853298e+00, -5.01175976e+00,  2.36719608e+00,
         2.18237359e-02,  1.07961506e-01,  1.13408017e+00,
         2.29868674e+00, -1.94324148e+00,  2.40340972e+00,
        -3.71920198e-01, -1.02654719e+00, -1.75946367e+00,
         3.86554807e-01,  2.38254476e+00,  2.45998216e+00,
         5.67573965e-01, -3.11678839e+00,  1.40924290e-01,
        -1.69877615e-02,  2.57056475e-01, -3.48681855e+00,
         2.74431658e+00,  1.25053751e+00, -3.04521608e+00,
        -1.90681434e+00,  1.85229897e-01,  1.43182397e-

In [138]:
# arr_predict = []
for diff_file in file_list:
# for layer in range(1,20):
    arr_predict = []
    train_df = pd.read_csv(diff_file,header=None)
#     train_df = pd.read_csv('/kaggle/input/'+str(layer)+'_'+str(seqorpar)+'_poly_perf.csv',header=None)
#     myname = str(layer)+'_'+str(seqorpar)+'_poly_perf.csv'
    myname = "results_"+diff_file
    train_df = train_df.sort_values(by=[1],ascending=False)
    print_ver_GF = train_df.dropna(axis=1)
    print_ver_GF = np.asarray(print_ver_GF)
#     print(print_ver_GF[0,1])
    train_df = train_df.drop([0,1,2,3,4,5,6,7,8,9], axis=1)
    train_df = train_df.dropna(axis=1)
    npdata = np.array(train_df)
    for i in range(len(npdata)):
        for j in range(len(npdata)):
            arr_predict.append(np.concatenate((npdata[i],npdata[j]), axis=0))
    arr_predict = np.asarray(arr_predict)
    prediction = model.predict(arr_predict)

#     print(str(layer)+'_'+str(seqorpar)+'_poly_perf.csv')
    print(diff_file)
    for x in prediction:
        print(x)
    break
    print(prediction[:,0])
    version =0
    flag=0
    count_win =0
    distinct_wins = []
    Actual_rank = 0
    print_result = []
    for values in prediction[:,0]:
        if flag<len(npdata):
            flag +=1
            if values>0.7:
                count_win+=1
        else:
            print_result.append([Actual_rank+1,print_ver_GF[Actual_rank,1],print_ver_GF[Actual_rank,0],count_win])
            Actual_rank+=1
            version+=1
            if count_win not in distinct_wins:
                distinct_wins.append(count_win)
            count_win=0
            flag=1
            if values>0.7:
                count_win+=1
    print_result.append([Actual_rank+1,print_ver_GF[Actual_rank,1],print_ver_GF[Actual_rank,0],count_win])
    if count_win not in distinct_wins:
        distinct_wins.append(count_win)

#     print(pd.DataFrame(print_result))
#     print(distinct_wins)
    distinct_wins.sort(reverse = True)
    polyrank = []
    for values in print_result:
        polyrank.append(distinct_wins.index(values[3])+1)
    polyrank = pd.DataFrame(polyrank)
#     print(pd.DataFrame(polyrank))
    Final = pd.concat([pd.DataFrame(print_result),polyrank], axis=1, sort=False)
    Final= np.asarray(Final)
#     print(Final)
#     print("Max GFlops",np.max(Final[:,1]))
    temp = Final[Final[:,4] == 1]
#     print("Top 1 GFLop by poly",np.max(temp[:,1]))
    num_row_in_top5_per = math.ceil((Final.shape[0])*0.05)
#     print(num_row_in_top5_per)
    rank = 1
    top5_max_list =[]
#     print(temp.shape)
#     print(temp)
    while num_row_in_top5_per>0:
        temp_rank = Final[Final[:,4] == rank]
        top5_max_list.append(np.max(temp[:,1]))
        num_row_in_top5_per-=temp_rank.shape[0]
        rank+=1
#     print("Final Max value in top 5 percent is",max(top5_max_list))
    top5_perf_list = [np.max(Final[:,1]),np.max(temp[:,1]),Final.shape[0],max(top5_max_list),np.min(Final[:,1]),np.median(Final[:,1])]
#     print(top5_perf_list)
    f = open(myname, "a+") 
    f.write('ActualRank, GFlops, Version, wins, PolyRank,\n')
    f.close()
    for values in Final:
        f = open(myname, "a+")        
        str_temp = ""
        for val in values:
            str_temp = str_temp + str(val)+", "
#         print(str_temp)
        f.write("%s\n" % str_temp)
        f.close()
    myname_summary = "summary_results_28"
    string2 = re.search("\w+/\w+/",diff_file).group(0)
    if string2 in diff_file:
        string2 = diff_file.replace(string2,'')
    if "_poly_perf.csv" in string2:
        string2 = string2.replace("_poly_perf.csv",'')
#     print(string2)
    f = open(myname_summary, "a")
#     f.write("Max_GFLOPS, Poly_Top_1GFLOPS,numVariants,Poly_Top_0.050000,Min_GFLOPS,Median_GFLOPS,\n")
    str_temp = string2+", "
    for val in top5_perf_list:
        str_temp = str_temp + str(val) + ", "
    f.write("%s\n" % str_temp)
    f.close()

gagan_Oct_24_2019_data/alexnet/alexnet_2_28_poly_perf.csv
[0.00451821 0.9954817 ]
[0.00451821 0.9954817 ]
[1.000000e+00 8.030658e-09]
[1.000000e+00 8.030627e-09]
[1.000000e+00 8.030658e-09]
[1.000000e+00 8.030688e-09]
[1.000000e+00 8.030688e-09]
[1.000000e+00 8.030704e-09]
[1.000000e+00 8.030688e-09]
[1.000000e+00 8.030658e-09]
[0.00451821 0.9954817 ]
[0.00451821 0.9954817 ]
[1.000000e+00 8.030658e-09]
[1.000000e+00 8.030627e-09]
[1.000000e+00 8.030658e-09]
[1.000000e+00 8.030688e-09]
[1.000000e+00 8.030688e-09]
[1.000000e+00 8.030704e-09]
[1.000000e+00 8.030688e-09]
[1.000000e+00 8.030658e-09]
[1.0277227e-08 1.0000000e+00]
[1.0277227e-08 1.0000000e+00]
[5.736727e-08 1.000000e+00]
[0.8849127  0.11508731]
[9.9999881e-01 1.1969523e-06]
[0.00362398 0.996376  ]
[0.00362398 0.996376  ]
[1.000000e+00 4.407361e-08]
[0.00362398 0.996376  ]
[9.9946982e-01 5.3014985e-04]
[3.2935874e-05 9.9996710e-01]
[3.2935874e-05 9.9996710e-01]
[0.0045182  0.99548185]
[5.7369896e-08 1.0000000e+00]
[1.409812e-0